# <center><font color=slate>Exceptions and errors</font></center>

## <center>Exceptions are arranged in an <font color=tomato>inheritance</font> hierarchy</center>
he built in exception classes are arranged into a class hierarchy using inheritance.
Example:

In [82]:
IndexError.mro(), KeyError.mro()

([IndexError, LookupError, Exception, BaseException, object],
 [KeyError, LookupError, Exception, BaseException, object])

These two exceptions are siblings, both are subclasses of `LookupError`.

In [83]:
def lookups():
    s = [1, 4, 6]
    try:
        item = s[5]
    except IndexError:
        print('Handled IndexError')

    d = dict(a=65, b=66, c=67)
    try:
        value = d['x']
    except KeyError:
        print('Handled KeyError')

lookups()

Handled IndexError
Handled KeyError


So, in practice we can catch both `IndexError` and `KeyError` exceptions by catching `LookupError`

In [84]:
def lookups():
    s = [1, 4, 6]
    try:
        item = s[5]
    except LookupError:
        print('Handled IndexError')

    d = dict(a=65, b=66, c=67)
    try:
        value = d['x']
    except LookupError:
        print('Handled KeyError')

lookups()

Handled IndexError
Handled KeyError


## <center>Full Hierarchy<font color=tomato><br>Built-in Exceptions</font></center>
https://docs.python.org/3/library/exceptions.html#exception-hierarchy

## <center>Exception<font color=tomato><br>payloads</font></center>


In [85]:
def median(iterable):
    items = sorted(iterable)
    median_index = (len(items) - 1) // 2
    if len(items) % 2 != 0:
        return items[median_index]
    return (items[median_index] + items[median_index + 1]) / 2.0

median([5, 2, 1, 7, 8, 0])

3.5

In [86]:
try:
    median([])
except Exception as e:
    print(e.__repr__())


IndexError('list index out of range')


In [87]:
def median(iterable):
    items = sorted(iterable)
    if len(items) == 0:
        raise ValueError("median() arg is empty sequence")
    median_index = (len(items) - 1) // 2
    if len(items) % 2 != 0:
        return items[median_index]
    return (items[median_index] + items[median_index + 1]) / 2.0


In [88]:
try:
    median([])
except Exception as e:
    print(e.__repr__(), e.args)


ValueError('median() arg is empty sequence') ('median() arg is empty sequence',)


Specific exception classes may provide additional specific named attributes,
which contain further information about the cause.

`UnicodeError` is one such example, which has five additional named:
 `attributes`, `encoding`, `reason`, `object`, `start`, and 1`

In [89]:
try:
    b'\x81'.decode('utf-8')
except UnicodeError as e:
    print(e)
    print('encoding: ', e.encoding)
    print('reason: ', e.reason)
    print('object: ', e.object)
    print('start: ', e.start)
    print('end: ', e.end)

'utf-8' codec can't decode byte 0x81 in position 0: invalid start byte
encoding:  utf-8
reason:  invalid start byte
object:  b'\x81'
start:  0
end:  1


## <center>Defining <font color=tomato>new </font>exceptions</center>


In [90]:
import math

In [91]:
def triangle_area(a, b, c):
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return a
triangle_area(3, 4, 5)

6.0

In [92]:
try:
    triangle_area(10, 3, 30)
except Exception as e:
    print(e.__repr__())

ValueError('math domain error')


Given the following example, rather than the obscure math domain error message here, we'd prefer to raise a more specific exception

First we define our own exception class,`TriangleError`, subclassing`Exception`,
all the functionality we want is inherited from`Exception`: `__init__()`, `__str__()` and `__repr__()`

In [93]:
class TriangleError(Exception):
    def __init__(self, text, sides):
        super(). __init__(text)
        self._sides = tuple(sides)

    @property
    def sides(self):
        return self._sides

    def __str__(self):
        return "'{}' for sides {}".format(self.args[0], self._sides)
    def __repr__(self):
        return "TriangleError {!r}, {!r}".format(self.args[0], self._sides)

def triangle_area(a, b, c):
    sides = sorted((a, b, c))
    #sorting the three sides we are given and check that the length of the longest side isn't greater than the length of the two shorter sides added together we find out that a triangle is not possible
    if sides[2] > sides[0] + sides[1]:
        raise TriangleError(text='Illegal triangle', sides=sides)
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return a

try:
    triangle_area(3, 4, 10)
except Exception as e:
    print(e), print('\n'), print(e.__repr__())

'Illegal triangle' for sides (3, 4, 10)


TriangleError 'Illegal triangle', (3, 4, 10)


## <center><font color=tomato>Chaining</font> exceptions</center>

Exception chaining allows us to associate one exception with another. And there's two main use cases:
-   <font color=mediumTurquoise>Implicit chaining:</font> when during processing of one exception, another exception occurs, usually in a way incidental to the first exception.

In [95]:
import sys
def implicitChaining():
    try:
        a = triangle_area(3, 4, 10)
        print(a)
    except TriangleError as e:
        print(e, file=sys.stdin)


executing the `implicitChaining()` function we get the following exceptions:

The Python runtime machinery associates the original exception with the new exception by setting the special `__context__` attribute of the most recent exception
catching the two errors would be like this:

In [96]:
import sys
from io import UnsupportedOperation

def implicitChaining():
    try:
        a = triangle_area(3, 4, 10)
        print(a)
    except TriangleError as e:
        try:
            print(e, file=sys.stdin)
        except UnsupportedOperation as f:
            print(e.__repr__())
            print(f.__repr__())
            print(f.__context__ is e)

implicitChaining()

TriangleError 'Illegal triangle', (3, 4, 10)
UnsupportedOperation('not writable')
True


-   <font color=mediumTurquoise>Explicit chaining:</font> when we wish to deliberately handle an exception by translating it into a different exception type.


In [97]:
import math

class InclinationError(Exception):
    pass

def inclination(dx, dy):
    try:
        return math.degrees(math.atan(dy / dx))
    except ZeroDivisionError as e:
        raise InclinationError("Slope cannot be vertical") from e

The `from e` suffix associates the new exceptional object with the original exception `e`

Explicit chaining associates the chained exception through the `__cause__` attribute.

In [101]:
try:
    inclination(0, 5)
except InclinationError as e:
    print(e.__repr__(), '\n', e.__cause__.__repr__())

InclinationError('Slope cannot be vertical') 
 ZeroDivisionError('division by zero')


## <center><font color=tomato>Tracebacks</font><br>`__traceback__`</center>
In Python 3, each exception has a `__traceback__` special attribute, which contains a reference to the traceback object associated with that exception.

There is a Python standard library, `traceback module`, which contains functions for interrogating traceback objects.

In [109]:
import traceback
try:
    inclination(0, 5)
except InclinationError as e:
    s = traceback.format_tb(e.__traceback__)
    print(e.__traceback__, '\n'), print(s, '\n', ), traceback.print_tb(e.__traceback__)


['  File "<ipython-input-109-c2f663a17ab0>", line 3, in <module>\n    inclination(0, 5)\n', '  File "<ipython-input-97-cab1e03c47fa>", line 10, in inclination\n    raise InclinationError("Slope cannot be vertical") from e\n'] 



  File "<ipython-input-109-c2f663a17ab0>", line 3, in <module>
    inclination(0, 5)
  File "<ipython-input-97-cab1e03c47fa>", line 10, in inclination
    raise InclinationError("Slope cannot be vertical") from e


## <center>Checking invariants<br>with <font color=tomato>Assertions</font></center>
The purpose of assertions is to help prevent bugs creeping into the code
## <center>`assert condition [, messages]`</center>
If the condition is <font color=mediumTurquoise>false</font>, an <font color=mediumTurquoise>AssertionError</font> exception is raised causing the program to terminate.

If the message is supplied, it is used as the exception payload.

The purpose of the `assertion` statement is to give a convenient means for monitoring program invariants, which are conditions which should always be true for the program.

Assertion should not be used to validate arguments for the function, only to detect if the implementation of the function is incorrect.

In [111]:
try:
    assert False, "The condition is false"
except Exception as e:
    print(e.__repr__())

AssertionError('The condition is false')


In [ ]:
def modulus_three(n):
    r = n % 3
    if r == 0:
        print('Multiple of 3')
    elif r == 1:
        print("Reminder 1")
    else: # r == 2
        print("Reminder 2")

instead of the comment `# r = 2`:

In [112]:
def modulus_three(n):
    r = n % 3
    if r == 0:
        print('Multiple of 3')
    elif r == 1:
        print("Reminder 1")
    else:
        assert r==2, "Reminder is not 2"
        print("Reminder 2")